In [1]:
import jax
print(jax.default_backend())

def jax_has_gpu():

    try:
        _ = jax.device_put(jax.numpy.ones(1), device=jax.devices('gpu')[0])
        return True
    except:
        return False

print(jax_has_gpu())

import haiku as hk

gpu
True


In [2]:
# set-up in config.py
# set grid dim in eval_fcts.py

from ml_tools import state_utils

from eval_fcts import init, sample_prior_grid, sample_conditional_grid, get_grid_data, batch_init

import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp

import os



SEED = 10

key = jax.random.PRNGKey(SEED)

direct = "logs/regression-Sep05-eval/Sep05_103110_swirl_xvwd"
key, skey = jax.random.split(key, 2)




state = init(batch_init, skey)


index = state_utils.find_latest_checkpoint_step_index(direct)
state = state_utils.load_checkpoint(state, direct, step_index=index)

2024-09-11 10:08:46.224390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


checkpoint 1
checkpoint 2
checkpoint 3
checkpoint 4
checkpoint 5


In [3]:
hk.data_structures.tree_size(state.params)

187138

In [ ]:
for i in range(20):
    SEED +=1

    pkey = jax.random.PRNGKey(SEED)

    X, Y = sample_prior_grid(state, pkey)

    x1 = jnp.reshape(X[:,0], (20,20))
    x2 = jnp.reshape(X[:,1], (20,20))

    u = jnp.reshape(Y[:,0], (20,20))
    v = jnp.reshape(Y[:,1], (20,20))

    M = jnp.hypot(u,v) # create matrix that has lengths of vectors, used for coloring

    fig, ax = plt.subplots(figsize=(10,6), dpi = 600)
    q = ax.quiver(x1, x2, u, v, M, pivot='tail', width=0.004, scale_units='x', scale=5)

    ax.set_aspect('equal')

    dir = "plots"
    name = f"swirl_prior_{i}"
    pathname = os.path.join(dir, name)
    plt.savefig(pathname)

    #plt.show()

In [3]:
%%capture
n = 80
data = jnp.load("data/swirl_2_training.npz")

for i in range(n):

    SEED += 1

    ikey = jax.random.PRNGKey(SEED)

    idx = i

    

    x = data["x_target"][idx]
    y = data["y_target"][idx]

    n_ctxt = (i // 20 + 1) * 4

    ind = jax.random.choice(ikey, jnp.arange(399), shape=(n_ctxt,))

    xc = x[ind]
    yc = y[ind]

    SEED += 1
    # change seed when you change direction!
    ckey = jax.random.PRNGKey(SEED)

    X, Y, xc, yc = sample_conditional_grid(state, ckey, xc, yc)

    #mse += jnp.sum(jnp.sum((y-Y)**2, axis = 1) / 2, axis = 0) / 400 / n

    x1 = jnp.reshape(X[:,0], (20,20))
    x2 = jnp.reshape(X[:,1], (20,20))

    u = jnp.reshape(Y[:,0], (20,20))
    v = jnp.reshape(Y[:,1], (20,20))

    M = jnp.hypot(u,v) # create matrix that has lengths of vectors, used for coloring


    fig, ax = plt.subplots(figsize=(10,6), dpi = 600)
    q = ax.quiver(x1, x2, u, v, M, width=0.004, scale_units='x', scale=5)

    for x, y in zip(xc, yc):
        ax.quiver(x[0], x[1], y[0], y[1], color='red', width=0.004, scale_units='x', scale=5,)

    ax.set_aspect('equal')

    dir = "plots"
    name = f"swirl_posterior_{i}"
    pathname = os.path.join(dir, name)
    plt.savefig(pathname)

    #plt.show()

